In [1]:
import os


In [2]:
# Корневая папка с отчетами
# NOTE: \\\\ в начале пути это двойное экранирование слэшей, TBC поменяется ли что-то при переезде на VM в итоге
network_folder = os.path.join("\\\\corp.suek.ru", "Share", "MSK_market_data", "market_data", "Market Reports")

# Нам интересны только конечные папки с файлами
folders_with_files = [
    (root, dirs, files)
    for (root, dirs, files) in os.walk(network_folder)
    if len(files) > 0
]
folders_with_files


[('\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Argus\\Argus Russian Coal\\2024\\10_October',
  [],
  ['Argus Russian Coal (2024-10-07).pdf']),
 ('\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Platts\\Coal Trader\\2024\\10_October',
  [],
  ['Coal Trader_10 Oct 2024.pdf']),
 ('\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Platts\\Coal Trader International\\2024\\10_October',
  [],
  ['Coal Trader International_10 Oct 2024.pdf'])]

In [3]:
# Для каждого вида отчета возьмём только последний вариант (TBC у нас ведь не спросят "какая ситуация была полгода назад"?)
# NOTE: сейчас на сетевом диске только по одному файлу каждого типа, но в будущем они будут докидываться
# Для этого опираемся на структуру папок:
# - последний уровень - месяц
# - предпоследний - год
# - пред-предпоследний - тип отчета

latest_reports = {}
for (root, dirs, files) in folders_with_files:
    report_type = root.split(os.sep)[-3]
    report_year = root.split(os.sep)[-2]
    report_month = root.split(os.sep)[-1]
    # Проверяем актуальность
    set_this_as_best = False
    previous_best = latest_reports.get(report_type)
    if previous_best is None:
        # Это первый репорт такого типа
        set_this_as_best = True
    else:
        # Нужно понять новее ли эта итерация
        if (report_year > previous_best["year"]) or ((report_year == previous_best["year"]) and (report_month > previous_best["month"])):
            set_this_as_best = True

    # Если поняли что нам это нужно
    if set_this_as_best:
        latest_reports[report_type] = {
            "year": report_year,
            "month": report_month,
            "files": [
                os.path.join(root, file) for file in files
            ],  # NOTE 2: TBC что делать если в папке будет несколько файлов - читаем все?
        }

# Соберём все в один список путей
files_to_read = sum([v["files"] for _, v in latest_reports.items()], [])

files_to_read


['\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Argus\\Argus Russian Coal\\2024\\10_October\\Argus Russian Coal (2024-10-07).pdf',
 '\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Platts\\Coal Trader\\2024\\10_October\\Coal Trader_10 Oct 2024.pdf',
 '\\\\corp.suek.ru\\Share\\MSK_market_data\\market_data\\Market Reports\\Platts\\Coal Trader International\\2024\\10_October\\Coal Trader International_10 Oct 2024.pdf']